In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc3_2_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc3'+"_2_32.csv", index_col=0)

# 형태 출력
print(gcc3_2_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc3_2_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc3_2_32.head()

(1749581, 2)
reset_index 완료
input data shape


,bin,label
0,55,1
1,89,0
2,e5,0
3,57,0
4,56,0


In [5]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc3_2_32['bin'].unique()))

# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc3_2_32_onehot = pd.get_dummies(gcc3_2_32)


print('원핫인코딩완료')

print(gcc3_2_32_onehot.shape)

# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc3_2_32_onehot['label'].value_counts())

256
원핫인코딩완료
(1749581, 257)
0    1746216
1       3365
Name: label, dtype: int64


In [11]:
# (5-1) gcc3 6gram
######################## 
idx3 = gcc3_2_32_onehot[gcc3_2_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 
left_idx3, right_idx3 = 0, 7 # 3개씩

# 6gram
for k in range(left_idx3, right_idx3):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능
print(len(ls_idx3))

#ls_idx3 = list(set(ls_idx3)) 
print(len(ls_idx3))

ls_idx3.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx3 = list(filter(lambda x: x<len(gcc3_2_32_onehot), ls_idx3))
print(len(ls_idx3))

# 2차 남은 index들 중 right_idx3 나눈 나머지 없애기
sub = len(ls_idx3)%(right_idx3)
print(sub)

ls_idx3 = ls_idx3[:len(ls_idx3)-sub]
print(len(ls_idx3))

print('gcc3_2_32', len(ls_idx3))

# loc 로 수정필요
gcc3_2_32_onehot_3gram = gcc3_2_32_onehot.loc[ls_idx3,:].copy()

23555
23555
23555
0
23555
gcc3_2_32 23555


In [12]:
# 훈련 데이터, 훈련 라벨
x_gcc3_2_32_3 = gcc3_2_32_onehot_3gram.iloc[:,1:].to_numpy()
y_gcc3_2_32_3 = gcc3_2_32_onehot_3gram['label'].to_numpy()
print(x_gcc3_2_32_3.shape, y_gcc3_2_32_3.shape)


(23555, 256) (23555,)


In [13]:
x_gcc3_2_32_3 = x_gcc3_2_32_3.reshape(-1, right_idx3, x_gcc3_2_32_3.shape[1])
y_gcc3_2_32_3 = y_gcc3_2_32_3.reshape(-1, right_idx3, 1)

print(x_gcc3_2_32_3.shape, y_gcc3_2_32_3.shape)

(3365, 7, 256) (3365, 7, 1)


## 모델

In [14]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx3, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 32)             34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 1)              33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


In [15]:
########## 3gram

# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc3_2_32_3, y_gcc3_2_32_3):
    print('======Training stage======')
    model1.fit(x_gcc3_2_32_3[train],
               y_gcc3_2_32_3[train],
               epochs = 30,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc3_2_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc3_2_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/30
3028/3028 [==============================] - 2s 590us/step - loss: 0.4276 - accuracy: 0.8385
Epoch 2/30
3028/3028 [==============================] - 1s 201us/step - loss: 0.1421 - accuracy: 0.9446
Epoch 3/30
3028/3028 [==============================] - 1s 199us/step - loss: 0.0461 - accuracy: 0.9949
Epoch 4/30
3028/3028 [==============================] - 1s 200us/step - loss: 0.0241 - accuracy: 0.9954
Epoch 5/30
3028/3028 [==============================] - 1s 206us/step - loss: 0.0169 - accuracy: 0.9962
Epoch 6/30
3028/3028 [==============================] - 1s 201us/step - loss: 0.0129 - accuracy: 0.9963
Epoch 7/30
3028/3028 [==============================] - 1s 205us/step - loss: 0.0100 - accuracy: 0.9966
Epoch 8/30
3028/3028 [==============================] - 1s 203us/step - loss: 0.0080 - accuracy: 0.9968
Epoch 9/30
3028/3028 [==============================] - 1s 207us/step - loss: 0.0066 - accuracy: 0.9969
Epoch 10/30
3028/3028 [==============

accuracy_score 0.9991521831284442
recall_score 0.9941348973607038
precision_score 1.0
f1_score 0.9970588235294118
======Training stage======
Epoch 1/30
3028/3028 [==============================] - 1s 221us/step - loss: 0.0024 - accuracy: 0.9997
Epoch 2/30
3028/3028 [==============================] - 1s 224us/step - loss: 0.0022 - accuracy: 0.9997
Epoch 3/30
3028/3028 [==============================] - 1s 215us/step - loss: 0.0021 - accuracy: 0.9998
Epoch 4/30
3028/3028 [==============================] - 1s 212us/step - loss: 0.0020 - accuracy: 0.9998
Epoch 5/30
3028/3028 [==============================] - 1s 214us/step - loss: 0.0019 - accuracy: 0.9998
Epoch 6/30
3028/3028 [==============================] - 1s 217us/step - loss: 0.0019 - accuracy: 0.9998
Epoch 7/30
3028/3028 [==============================] - 1s 217us/step - loss: 0.0018 - accuracy: 0.9998
Epoch 8/30
3028/3028 [==============================] - 1s 223us/step - loss: 0.0017 - accuracy: 0.9999
Epoch 9/30
3028/3028 [=====

accuracy_score 0.9995760915642221
recall_score 0.9970414201183432
precision_score 1.0
f1_score 0.9985185185185186
======Training stage======
Epoch 1/30
3028/3028 [==============================] - 1s 257us/step - loss: 0.0011 - accuracy: 0.9999
Epoch 2/30
3028/3028 [==============================] - 1s 262us/step - loss: 0.0011 - accuracy: 0.9999
Epoch 3/30
3028/3028 [==============================] - 1s 264us/step - loss: 0.0011 - accuracy: 0.9999
Epoch 4/30
3028/3028 [==============================] - 1s 263us/step - loss: 0.0010 - accuracy: 0.9999
Epoch 5/30
3028/3028 [==============================] - 1s 266us/step - loss: 0.0010 - accuracy: 0.9999
Epoch 6/30
3028/3028 [==============================] - 1s 254us/step - loss: 0.0010 - accuracy: 0.9999
Epoch 7/30
3028/3028 [==============================] - 1s 264us/step - loss: 0.0011 - accuracy: 0.9999
Epoch 8/30
3028/3028 [==============================] - 1s 264us/step - loss: 0.0010 - accuracy: 0.9999
Epoch 9/30
3028/3028 [=====

accuracy_score 0.9995760915642221
recall_score 1.0
precision_score 0.9970674486803519
f1_score 0.9985315712187959
======Training stage======
Epoch 1/30
3028/3028 [==============================] - 1s 284us/step - loss: 7.8811e-04 - accuracy: 0.9999
Epoch 2/30
3028/3028 [==============================] - 1s 278us/step - loss: 8.0705e-04 - accuracy: 0.9999
Epoch 3/30
3028/3028 [==============================] - 1s 298us/step - loss: 7.7881e-04 - accuracy: 0.9999
Epoch 4/30
3028/3028 [==============================] - 1s 278us/step - loss: 8.1470e-04 - accuracy: 0.9999
Epoch 5/30
3028/3028 [==============================] - 1s 286us/step - loss: 7.6689e-04 - accuracy: 0.9999
Epoch 6/30
3028/3028 [==============================] - 1s 298us/step - loss: 7.6716e-04 - accuracy: 0.9999
Epoch 7/30
3028/3028 [==============================] - 1s 299us/step - loss: 7.2846e-04 - accuracy: 0.9999
Epoch 8/30
3028/3028 [==============================] - 1s 312us/step - loss: 7.3659e-04 - accuracy: 0.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
3028/3028 [==============================] - 1s 347us/step - loss: 3.0580e-04 - accuracy: 0.9999
Epoch 2/30
3028/3028 [==============================] - 1s 359us/step - loss: 3.0878e-04 - accuracy: 0.9999
Epoch 3/30
3028/3028 [==============================] - 1s 341us/step - loss: 2.7572e-04 - accuracy: 0.9999
Epoch 4/30
3028/3028 [==============================] - 1s 331us/step - loss: 2.4645e-04 - accuracy: 0.99990s - los
Epoch 5/30
3028/3028 [==============================] - 1s 339us/step - loss: 2.6417e-04 - accuracy: 0.9999
Epoch 6/30
3028/3028 [==============================] - 1s 344us/step - loss: 2.4738e-04 - accuracy: 0.9999
Epoch 7/30
3028/3028 [==============================] - 1s 335us/step - loss: 2.1468e-04 - accuracy: 0.9999
Epoch 8/30
3028/3028 [==============================] - 1s 335us/step - loss: 1.8684e-04 - accuracy: 0.9999
Epoch 9/30
3028/3028 [==========

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
3029/3029 [==============================] - 1s 328us/step - loss: 2.3807e-07 - accuracy: 1.00001s - l
Epoch 2/30
3029/3029 [==============================] - 1s 300us/step - loss: 1.8350e-07 - accuracy: 1.00000s - loss: 1.8339e-07 - accuracy
Epoch 3/30
3029/3029 [==============================] - 1s 332us/step - loss: 1.4673e-07 - accuracy: 1.0000
Epoch 4/30
3029/3029 [==============================] - 1s 328us/step - loss: 1.2260e-07 - accuracy: 1.0000
Epoch 5/30
3029/3029 [==============================] - 1s 335us/step - loss: 1.0521e-07 - accuracy: 1.0000
Epoch 6/30
3029/3029 [==============================] - 1s 349us/step - loss: 9.0600e-08 - accuracy: 1.0000
Epoch 7/30
3029/3029 [==============================] - 1s 343us/step - loss: 7.8255e-08 - accuracy: 1.00000s - los
Epoch 8/30
3029/3029 [==============================] - 1s 340us/step - loss: 6.8922e-08 - accuracy: 1

accuracy_score 0.9995748299319728
recall_score 0.9970414201183432
precision_score 1.0
f1_score 0.9985185185185186
======Training stage======
Epoch 1/30
3029/3029 [==============================] - 1s 348us/step - loss: 9.5563e-05 - accuracy: 1.0000
Epoch 2/30
3029/3029 [==============================] - 1s 361us/step - loss: 5.0490e-05 - accuracy: 1.00000s - loss: 4.342 - ETA: 0s - loss: 6.9242e-05 - accu
Epoch 3/30
3029/3029 [==============================] - 1s 349us/step - loss: 3.0498e-05 - accuracy: 1.0000
Epoch 4/30
3029/3029 [==============================] - 1s 338us/step - loss: 1.3353e-05 - accuracy: 1.0000
Epoch 5/30
3029/3029 [==============================] - 1s 393us/step - loss: 9.7193e-06 - accuracy: 1.00000s - loss: 5.0843e-05 - accuracy - ETA: 0s - loss: 2
Epoch 6/30
3029/3029 [==============================] - 1s 419us/step - loss: 7.1055e-06 - accuracy: 1.0000
Epoch 7/30
3029/3029 [==============================] - 1s 392us/step - loss: 7.5406e-06 - accuracy: 1.0000

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
3029/3029 [==============================] - 1s 331us/step - loss: 2.0978e-07 - accuracy: 1.0000
Epoch 2/30
3029/3029 [==============================] - 1s 327us/step - loss: 1.8278e-07 - accuracy: 1.0000
Epoch 3/30
3029/3029 [==============================] - 1s 332us/step - loss: 1.6123e-07 - accuracy: 1.0000
Epoch 4/30
3029/3029 [==============================] - 1s 336us/step - loss: 1.7151e-07 - accuracy: 1.0000
Epoch 5/30
3029/3029 [==============================] - 1s 335us/step - loss: 1.4685e-07 - accuracy: 1.0000
Epoch 6/30
3029/3029 [==============================] - 1s 337us/step - loss: 1.4687e-07 - accuracy: 1.0000
Epoch 7/30
3029/3029 [==============================] - 1s 345us/step - loss: 1.2542e-07 - accuracy: 1.0000
Epoch 8/30
3029/3029 [==============================] - 1s 321us/step - loss: 1.2743e-07 - accuracy: 1.0000
Epoch 9/30
3029/3029 [==================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
======Training stage======
Epoch 1/30
3029/3029 [==============================] - 1s 327us/step - loss: 4.3243e-08 - accuracy: 1.0000
Epoch 2/30
3029/3029 [==============================] - 1s 298us/step - loss: 4.3918e-08 - accuracy: 1.0000
Epoch 3/30
3029/3029 [==============================] - ETA: 0s - loss: 4.3911e-08 - accuracy: 1.00 - 1s 276us/step - loss: 4.2909e-08 - accuracy: 1.0000
Epoch 4/30
3029/3029 [==============================] - 1s 276us/step - loss: 4.0921e-08 - accuracy: 1.0000
Epoch 5/30
3029/3029 [==============================] - 1s 274us/step - loss: 3.7043e-08 - accuracy: 1.0000
Epoch 6/30
3029/3029 [==============================] - 1s 290us/step - loss: 3.6225e-08 - accuracy: 1.0000
Epoch 7/30
3029/3029 [==============================] - 1s 277us/step - loss: 3.4082e-08 - accuracy: 1.0000
Epoch 8/30
3029/3029 [==============================] - 1s 277us/step - loss: 3.5786e-08 - accuracy: 1

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9991521831284442, 0.9995760915642221, 0.9995760915642221, 1.0, 1.0, 1.0, 0.9995748299319728, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9941348973607038, 0.9970414201183432, 1.0, 1.0, 1.0, 1.0, 0.9970414201183432, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [1.0, 1.0, 0.9970674486803519, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9970588235294118, 0.9985185185185186, 0.9985315712187959, 1.0, 1.0, 1.0, 0.9985185185185186, 1.0, 1.0, 1.0]


In [17]:
# 4gram 평가지표
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997879196188861
10-Fold Cross_validation. Recall : 0.998821773759739
10-Fold Cross_validation. Precision : 0.9997067448680351
10-Fold Cross_validation. F1-Score : 0.9992627431785245
